<a href="https://colab.research.google.com/github/Maleniski/ing-caracteristicas/blob/main/DescargaDatosExploracionXml_MariaElenaMartinezManzanares.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>



# Curso *Ingeniería de Características*

### Descargando datos

Estudiante: María Elena Martínez Manzanares.

Programa: Maestría en Ciencia de Datos.

Asignatura: Ingeniería de Características.

Profesor: Julio Waissman Vilanova.

<p>
<img src="https://identidadbuho.unison.mx/wp-content/uploads/2019/06/letragrama-cmyk-72.jpg" width="150">
</p>


<a target="_blank" href="https://colab.research.google.com/github/mcd-unison/ing-caract/blob/main/ejemplos/integracion/python/descarga_datos.ipynb"><img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Ejecuta en Google Colab</a>

</center>

In [ ]:

import os  # Para manejo de archivos y directorios
import urllib.request # Una forma estandard de descargar datos
# import requests # Otra forma no de las librerías de uso comun
import xml.etree.ElementTree as et

import datetime # Fecha de descarga
import pandas as pd # Solo para ver el archivo descargado
import numpy as np
import zipfile # Descompresión de archivos

# Archivos xml

Los archivos *xml* son una manera de compartir información a través de internet o de guardar información con formatos genéricos que sigue siendo muy utilizada hoy en día. En general lidiar con archivos xml es una pesadilla y se necesita explorarlos con calma y revisarlos bien antes de usarlos. 

La definición del formato y su uso de pede revisar en [este tutorial de la w3schools](https://www.w3schools.com/xml/default.asp).


Wikipedia es un buen ejemplo de un lugar donde la información se guarda y se descarga en forma de archivos xml. Por ejemplo, si queremos descargar datos de la wikipedia [con su herramienta de exportación](https://es.wikipedia.org/wiki/Especial:Exportar) utilizando [las categorias definidas por Wikipedia](https://es.wikipedia.org/wiki/Portal:Portada). Para hacerlo en forma programática es ecesario usar la [API de Mediawiki](https://github.com/mudroljub/wikipedia-api-docs) que veremos más adelante.

Por el momento descargemos unos datos de *wikipedia* y hagamos el ejercicio de tratar de entender la estructura del árbol.

In [ ]:
archivo_url = "https://github.com/mcd-unison/ing-caract/raw/main/ejemplos/integracion/ejemplos/wikipedia-poetas.xml"
archivo_nombre = "poetas.xml"
subdir = "./data/"

if not os.path.exists(subdir + archivo_nombre):
    if not os.path.exists(subdir):
        os.makedirs(subdir)
    urllib.request.urlretrieve(archivo_url, subdir + archivo_nombre)


Creamos el acceso que nos permite navegar entre los bloques de información del archivo xml.

In [ ]:
poetas = et.parse(subdir + archivo_nombre)
root = poetas.getroot()

Desplegamos los atributos y vemos el nombre que nos permiten acceder al bloque de información. Podemos ver que para poder acceder a diferentes bloques de información (atributos del xml) tiene la estructura



> '{ http://www.mediawiki.org/xml/export-0.10/ }NombreDelAtributo'



In [ ]:
[elem.tag for elem in root.iter()][50:60]

['{http://www.mediawiki.org/xml/export-0.10/}id',
 '{http://www.mediawiki.org/xml/export-0.10/}comment',
 '{http://www.mediawiki.org/xml/export-0.10/}model',
 '{http://www.mediawiki.org/xml/export-0.10/}format',
 '{http://www.mediawiki.org/xml/export-0.10/}text',
 '{http://www.mediawiki.org/xml/export-0.10/}sha1',
 '{http://www.mediawiki.org/xml/export-0.10/}page',
 '{http://www.mediawiki.org/xml/export-0.10/}title',
 '{http://www.mediawiki.org/xml/export-0.10/}ns',
 '{http://www.mediawiki.org/xml/export-0.10/}id']

Accedemos al atributo autor, su biografía, fechas de modificación del artículo de wikipedia, el nombre del usuario que editó el artículo.

Ponemos esta información en una tabla de pandas.

In [ ]:
autores = []
for autor in root.iter('{http://www.mediawiki.org/xml/export-0.10/}title'):
    autores.append(autor.text)

In [ ]:
fechas_modificacion = []
for revision in root.iter('{http://www.mediawiki.org/xml/export-0.10/}timestamp'):
    fechas_modificacion.append(revision.text)

In [ ]:
biography = []
for bio in root.iter('{http://www.mediawiki.org/xml/export-0.10/}text'):
    biography.append(bio.text)

In [ ]:
username=[]
for usname in root.iter('{http://www.mediawiki.org/xml/export-0.10/}username'):
    username.append(usname.text)

In [ ]:
data = pd.DataFrame(columns = ['Autor','Biografia','Fecha_Modificacion'])
for i in range(len(autores)):
  data = pd.concat([data,
                  pd.DataFrame.from_records([{'Autor': autores[i], 
                                              'Biografia': biography[i], 
                                              "Fecha_Modificacion": fechas_modificacion[i]}]
                                            )], ignore_index=True)

Podemos acceder a más información si extraemos datos de la biografía. Dado que el bloque biografía es una cadena de caracteres, para poder extraer información es necesario utilizar expresiones regulares.

Podemos extraer, por ejemplo, el año de nacimiento, y el mes y día de nacimiento.

In [ ]:
import re

In [ ]:
year_birth=[]
day_month_birth=[]
for i in range(len(data)):
  
  try:
    year_birth.append(re.search(r"\[\[([1-3][0-9]{3})\]\]", data["Biografia"].iloc[i]).group(1))
  except:
    year_birth.append(np.nan)
  
  try:
    day_month_birth.append(re.search(r"\[\[([1-3][1-9]\s.*?)\]\]", data["Biografia"].iloc[i]).group(1))
  except:
    day_month_birth.append(np.nan)

In [ ]:
data["Año_Nacimiento"]=year_birth
data["Dia_Mes_Nacimiento"]=day_month_birth

In [ ]:
data

,Autor,Biografia,Fecha_Modificacion,Año_Nacimiento,Dia_Mes_Nacimiento
0,Julia Morilla de Campbell,'''Julia Morilla de Campbell''' ([[Rosario (Ar...,2017-10-30T07:46:11Z,1922,18 de julio
1,Luis Negreti,{{Ficha de escritor\n|Imagen = NE...,2016-09-17T17:35:26Z,1890,16 de enero
2,Poldy Bird,{{Ficha de persona\n| padres = Enrique Bird Mo...,2018-06-11T04:34:41Z,1941,16 de diciembre
3,Ana María Shua,{{Ficha de persona\n|imagen=\n|nombre de nacim...,2018-06-13T18:50:17Z,1951,22 de abril
4,León Benarós,{{Ficha de persona\n|nombre = León B...,2018-04-29T21:49:56Z,1915,25 de agosto
...,...,...,...,...,...
634,Humberto Tejera,{{Ficha de persona\n| nombre = Humberto Tejera...,2018-07-06T17:07:38Z,NaN,NaN
635,Mario Molina Cruz,{{Ficha de escritor\n|nombre= Mario Molina Cru...,2018-07-23T14:27:14Z,1955,19 de enero
636,Luis Ignacio Helguera,{{Ficha de persona\n| nombre = Luis Ignacio He...,2018-05-11T14:19:35Z,1962,11 de mayo
637,Daniel Olivares Viniegra,{{Promocional|8|julio}}\n\n'''Daniel Olivares ...,2018-07-20T21:56:20Z,NaN,NaN


Con esta información a la cual pudimos acceder, algunas preguntas interesantes seria ver si hay una concentración de nacimiento de poetas en alguna decada en particular.

También, podríamos analizar si hay concentración de fechas de moficación en un periodo en del tiempo en particular. Si esto fuera cierto, podría analizarse si esta relacionado con algún evento o suceso ocurrido en ese periodo.

Exportamos en parquet.

In [ ]:
data.to_parquet('poetas.parquet.gzip',compression='gzip')